In [21]:
MOD = 10**9 + 7

# Предварительно считаем факториалы и обратные факториалы
def precompute_factorials(n, mod):
    fact = [1] * (n + 1)
    inv_fact = [1] * (n + 1)
    for i in range(2, n + 1):
        fact[i] = fact[i - 1] * i % mod
    inv_fact[n] = pow(fact[n], mod - 2, mod)
    for i in range(n - 1, 0, -1):
        inv_fact[i] = inv_fact[i + 1] * (i + 1) % mod
    return fact, inv_fact

# DFS для подсчёта количества топологических сортировок
def dfs(u, tree, fact, inv_fact):
    dp_u = 1
    total_size = 0  # Количество всех вершин в поддереве
    sizes = []      # Размеры поддеревьев
    
    for v in tree[u]:
        dp_v, size_v = dfs(v, tree, fact, inv_fact)
        dp_u = dp_u * dp_v % MOD
        total_size += size_v
        sizes.append(size_v)
    
    # Учитываем перестановки всех вершин
    multinomial_coeff = fact[total_size]
    for size in sizes:
        multinomial_coeff = multinomial_coeff * inv_fact[size] % MOD
    
    dp_u = dp_u * multinomial_coeff % MOD
    return dp_u, total_size + 1

# Основная функция
def count_topological_sorts(n, edges):
    # Построение дерева
    tree = [[] for _ in range(n + 1)]
    is_child = [False] * (n + 1)
    for u, v in edges:
        tree[u].append(v)
        is_child[v] = True
    
    # Находим корень дерева
    root = next(i for i in range(1, n + 1) if not is_child[i])
    
    # Предварительный подсчёт факториалов
    fact, inv_fact = precompute_factorials(n, MOD)
    
    # Запуск DFS от корня
    result, _ = dfs(root, tree, fact, inv_fact)
    return result

# Чтение данных из файла
def main():
    with open("input.txt", "r") as f:
        n = int(f.readline().strip())  # Считываем число вершин
        edges = []
        for _ in range(n - 1):
            u, v = map(int, f.readline().strip().split())
            edges.append((u, v))
    
    # Вычисляем количество топологических сортировок
    result = count_topological_sorts(n, edges)
    
    # Записываем ответ в файл
    with open("output.txt", "w") as f:
        f.write(str(result) + "\n")

if __name__ == "__main__":
    main()


In [ ]:
from collections import defaultdict, deque

MOD = 10**9 + 7

def count_topological_sorts_dag(n, edges):

    graph = defaultdict(list)
    in_degree = [0] * (n + 1)
    for u, v in edges:
        graph[u].append(v)
        in_degree[v] += 1


    zero_in_degree = deque([i for i in range(1, n + 1) if in_degree[i] == 0])

  
    def dfs_count(zero_in_degree, current_count):
        if len(current_count) == n:
            return 1

        total_count = 0
        for i, node in enumerate(zero_in_degree):
            next_zero_in_degree = deque(zero_in_degree)
            next_zero_in_degree.remove(node)

            for neighbor in graph[node]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    next_zero_in_degree.append(neighbor)


            total_count += dfs_count(next_zero_in_degree, current_count + [node])
            total_count %= MOD

            for neighbor in graph[node]:
                if in_degree[neighbor] == 0:
                    next_zero_in_degree.remove(neighbor)
                in_degree[neighbor] += 1

        return total_count

    return dfs_count(zero_in_degree, [])

def main():

    with open("input.txt", "r") as f:
        n = int(f.readline().strip())
        edges = []
        for _ in range(n - 1):
            u, v = map(int, f.readline().strip().split())
            edges.append((u, v))
    
    result = count_topological_sorts_dag(n, edges)
    
    with open("output.txt", "w") as f:
        f.write(str(result) + "\n")

if __name__ == "__main__":
    main()


In [33]:
from collections import defaultdict, deque
MOD = 10**9 + 7

def count_topological_sorts_dag(n, edges):
    # Строим граф и определяем степени входа
    graph = defaultdict(list)
    in_degree = [0] * (n + 1)
    for u, v in edges:
        graph[u].append(v)
        in_degree[v] += 1
    
    # Ищем все вершины с нулевым входом
    zero_in_degree = deque([i for i in range(1, n + 1) if in_degree[i] == 0])
    
    # Используем кэширование для динамического программирования
    memo = {}

    # Функция для вычисления числа топологических сортировок с мемоизацией
    def dfs_count(zero_in_degree):
        if len(zero_in_degree) == 0:
            return 1

        # Преобразуем очередь в кортеж для использования в мемоизации
        key = tuple(zero_in_degree)
        if key in memo:
            return memo[key]
        
        total_count = 0
        # Пробуем все возможные вершины для выбора
        for i, node in enumerate(zero_in_degree):
            next_zero_in_degree = list(zero_in_degree)
            next_zero_in_degree.remove(node)

            # Обновляем входные степени для соседей
            for neighbor in graph[node]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    next_zero_in_degree.append(neighbor)

            total_count += dfs_count(next_zero_in_degree)
            total_count %= MOD

            # Восстанавливаем входные степени для соседей
            for neighbor in graph[node]:
                if in_degree[neighbor] == 0:
                    next_zero_in_degree.remove(neighbor)
                in_degree[neighbor] += 1

        memo[key] = total_count
        return total_count

    return dfs_count(zero_in_degree)

def main():
    with open("input.txt", "r") as f:
        n = int(f.readline().strip())
        edges = []
        for _ in range(n - 1):
            u, v = map(int, f.readline().strip().split())
            edges.append((u, v))
    
    result = count_topological_sorts_dag(n, edges)
    
    with open("output.txt", "w") as f:
        f.write(str(result) + "\n")

if __name__ == "__main__":
    main()


In [17]:
import sys

sys.setrecursionlimit(1000000000)
from collections import defaultdict
MOD = 10**9 + 7
def count_topological_sorts(n, edges):
    graph = defaultdict(list)
    in_degree = [0] * (n + 1)
    for u, v in edges:
        graph[u].append(v)
        in_degree[v] += 1
    in_degree_0 = [i for i in range(1, n + 1) if in_degree[i] == 0]
    dp = {}
    def dfs(in_degree_zero, in_degree):
        if len(in_degree_zero) == 0:
            return 1
        key = tuple(in_degree_zero)
        if key in dp:
            return dp[key]
        total_count = 0
        for i, node in enumerate(in_degree_zero):
            next_in_degree_zero = in_degree_zero[:i] + in_degree_zero[i+1:]
            for neigh in graph[node]:
                in_degree[neigh] -= 1
                if in_degree[neigh] == 0:
                    next_in_degree_zero.append(neigh)
            total_count += dfs(next_in_degree_zero, in_degree)
            total_count %= MOD
            for neigh in graph[node]:
                in_degree[neigh] += 1
        dp[key] = total_count
        return total_count
    return dfs(in_degree_0, in_degree)
def main():
    with open("input.txt", "r") as f:
        n = int(f.readline().strip())
        tree = []
        for _ in range(n - 1):
            u, v = map(int, f.readline().strip().split())
            tree.append((u, v))
    number = count_topological_sorts(n, tree)
    with open("output.txt", "w") as f:
        f.write(str(number) + "\n")
if __name__ == "__main__":
    main()


In [40]:
from collections import defaultdict, deque

MOD = 10**9 + 7

def solve():
    # Считывание данных из файла
    with open("input.txt", "r") as file:
        data = file.read().splitlines()
    
    N = int(data[0])
    edges = [tuple(map(int, line.split())) for line in data[1:N]]
    
    # Построим граф и посчитаем входящие и исходящие степени
    graph = defaultdict(list)
    indegree = [0] * (N + 1)
    outdegree = [0] * (N + 1)
    
    for a, b in edges:
        graph[a].append(b)
        indegree[b] += 1
        outdegree[a] += 1
    
    # Инициализируем DP и очередь для BFS
    dp = [0] * (N + 1)
    q = deque()
    
    # Листья имеют dp = 1
    for i in range(1, N + 1):
        if outdegree[i] == 0:
            dp[i] = 1
            q.append(i)
    
    # Обратный порядок обработки вершин
    while q:
        v = q.popleft()
        for parent in graph[v]:
            outdegree[parent] -= 1
            
            if outdegree[parent] == 0:
                q.append(parent)
            print(q)
            # Обновляем dp для родителя
            dp[parent] = (dp[parent] + dp[v]) % MOD
    
    # Найдем корень дерева (indegree == 0)
    root = next(i for i in range(1, N + 1) if indegree[i] == 0)
    print(root)
    # Записываем результат в файл
    with open("output.txt", "w") as file:
        file.write(f"{dp[root]}\n")
solve()

1
